## Extract ##
Realizando a etapa de extratação dos dados a partir do arquivo .csv. Para cada ID, será realizado uma requisição GET para obter os dados do usuário correspondente.

In [19]:
url_api = 'http://127.0.0.1:8000'

In [20]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')

user_id = df['UserID'].tolist()

print(user_id)

[4, 5, 6]


In [26]:
import requests
import json

def get_user(id):
    response = requests.get(f'{url_api}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in user_id if (user := get_user(id)) is not None]

print(json.dumps(users, indent=2))

[
  {
    "id": 4,
    "name": "Usuario1",
    "account": {
      "number": "00001-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "number": "**** *** *** 1111",
      "limit": 1000.0
    },
    "news": [
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Usuario1, seu futuro financeiro come\u00e7a hoje! Fa\u00e7a seu dinheiro trabalhar por voc\u00ea."
      }
    ]
  },
  {
    "id": 5,
    "name": "Usuario2",
    "account": {
      "number": "00002-2",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "number": "**** *** *** 2222",
      "limit": 1000.0
    },
    "news": [
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Usuario2, seu futuro merece mais! Invista e veja seus sonhos se realizarem. Comece hoje!"
      }
    ]


### Transformação ###

Utilizando API Google Gemini

In [ ]:
!python -m pip install --upgrade google-genai


In [29]:
import os
from google import genai
from google.genai import types

api_key = os.getenv("GEMINI_API_KEY")
client = genai.Client()

def generate_ai_news(user):
    completion = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        system_instruction="Você é um especialista de marketing que trabalha faz anos para uma agência bancária."
    ),
    contents=f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos. A mensagem deve ser feita levando em consideração o perfil de cliente baseado no seu saldo em conta de {user['account']['balance']} reais e seu limite do cartão de {user['card']['limit']} reais, porém, não fale de forma explícita o valor que o cliente tem ou insinuar que ele tem pouco ou muito dinheiro. A mensagem não deve ter mais de 100 caracteres. "
    )
    return completion.text

for user in users:
    news = generate_ai_news(user)
    print(news)
    user.setdefault("news", []).append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })


Usuario1, faça seu dinheiro crescer! Invista para construir um futuro financeiro sólido.
Usuario2, invista no seu futuro! Cada valor conta para seus sonhos.


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 32.748711018s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.

## Load ##
Atualizando a lista com o news para cada usuário da API.

In [24]:
def update_user(user):
    response = requests.put(f'{url_api}/users/{user['id']}', json=user)
    return True if response.status_code == 200 else False

for user in users:
    success = update_user(user)
    print(f"User {user['name']}  updated?  {success}")

User Usuario1  updated?  True
User Usuario2  updated?  True
User Usuario3  updated?  True
